In [0]:
PROJECT_PATH = '/home/dobos/project/ga_isochrones/python:' + \
    '/home/dobos/project/ga_pfsspec_all/python:' + \
    '/home/dobos/project/pysynphot'

GRID_PATH = '/datascope/subaru/data/pfsspec/models/stellar/grid/phoenix/phoenix_HiRes'
FILTER_PATH = '/datascope/subaru/data/pfsspec/subaru/hsc/filters/fHSC-g.txt'

ARMS = [ 'b', 'r', 'mr', 'n' ]
FIT_ARMS = [ 'b', 'mr', 'n' ]

DETECTOR_PATH = '/datascope/subaru/data/pfsspec/subaru/pfs/arms/{}.json'
DETECTORMAP_PATH = '/datascope/subaru/data/pfsspec/drp_pfs_data/detectorMap/detectorMap-sim-{}1.fits'
PSF_PATH = '/datascope/subaru/data/pfsspec/subaru/pfs/psf/import/{}.2'
SKY_PATH = '/datascope/subaru/data/pfsspec/subaru/pfs/noise/import/sky.see/{}/sky.h5'
MOON_PATH = '/datascope/subaru/data/pfsspec/subaru/pfs/noise/import/moon/{}/moon.h5'

XSHOOTER_PATH = '/datascope/subaru/data/catalogs/xshooter/'

# Radial Velocity fit

Demo code to perform maximum likelihood analysis of
radial velociy measurements between a spectrum and a
given template, drawn from the BOSZ models.

In [0]:
import os
import sys
import logging
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, MultipleLocator
from scipy.interpolate import interp1d
import h5py as h5
from tqdm.notebook import trange, tqdm

In [0]:
plt.rc('font', size=7)

In [0]:
# Allow load project as module
for p in reversed(PROJECT_PATH.split(':')):
    sys.path.insert(0, p)

os.environ['PYTHONPATH'] = PROJECT_PATH.split(':')[0]

In [0]:
os.environ['PYTHONPATH']

In [0]:
if 'debugpy' not in sys.modules:
    import debugpy
    debugpy.listen(("localhost", 5683))

In [0]:
%load_ext autoreload

In [0]:
%autoreload 2

# Load everything needed for the simulation

In [0]:
from pfs.ga.pfsspec.core.grid import ArrayGrid
from pfs.ga.pfsspec.stellar.grid import ModelGrid
from pfs.ga.pfsspec.stellar.grid.bosz import Bosz
from pfs.ga.pfsspec.stellar.grid.phoenix import Phoenix

from pfs.ga.pfsspec.core import Filter
from pfs.ga.pfsspec.sim.obsmod.pipelines import StellarModelPipeline
from pfs.ga.pfsspec.core import Physics
from pfs.ga.pfsspec.core.obsmod.psf import GaussPsf, PcaPsf
from pfs.ga.pfsspec.sim.obsmod.detectors import PfsDetector
from pfs.ga.pfsspec.sim.obsmod.detectormaps import PfsDetectorMap
from pfs.ga.pfsspec.sim.obsmod.background import Sky
from pfs.ga.pfsspec.sim.obsmod.background import Moon
from pfs.ga.pfsspec.core.obsmod.snr import QuantileSnr

from pfs.ga.pfsspec.stellar import StellarSpectrum
from pfs.ga.pfsspec.sim.obsmod.observations import PfsObservation
from pfs.ga.pfsspec.sim.obsmod.noise import NormalNoise
from pfs.ga.pfsspec.sim.obsmod.calibration import FluxCalibrationBias
from pfs.ga.pfsspec.core.obsmod.resampling import FluxConservingResampler

from pfs.ga.pfsspec.stellar.rvfit import RVFit, RVFitTrace
from pfs.ga.pfsspec.sim.rvfit import ModelGridRVFitSim

from pfs.ga.pfsspec.core.sampling import Parameter, ParameterSampler
from pfs.ga.pfsspec.sim.stellar import ModelGridSampler

### Spectrum grid

In [0]:
fn = os.path.join(GRID_PATH, 'spectra.h5')
grid = ModelGrid(Phoenix(), ArrayGrid)
grid.preload_arrays = False
grid.load(fn, format='h5')

### Detector config

In [0]:
detector = {}

for arm in ARMS:
    detector[arm] = PfsDetector()
    detector[arm].load_json(DETECTOR_PATH.format(arm))
    detector[arm].map = PfsDetectorMap()
    detector[arm].map.load(DETECTORMAP_PATH.format(arm[0]))

    # print(arm, detector[arm].map.default_fiberid)
    # print(arm, detector[arm].map.get_wave()[0].shape, detector[arm].map.get_wave()[0][[0, -1]], detector[arm].wave)

### Load detector LSF and precompute PCA kernel for high resolution

In [0]:
gauss_psf = {}
pca_psf = {}
template_psf = {}

for arm in ARMS:
    gauss_psf[arm] = GaussPsf()
    gauss_psf[arm].load(os.path.join(PSF_PATH.format(arm), 'gauss.h5'))

    # print(f'mean pixel size for arm {arm}', np.diff(detector[arm].get_wave()[0]).mean())
    # print(f'mean sigma and FWHM for arm {arm}', gauss_psf[arm].sigma.mean(), 2.355 * gauss_psf[arm].sigma.mean())

    s = gauss_psf[arm].get_optimal_size(grid.wave)
    # print(f'optimal kernel size for arm {arm}:', s)

    pca_psf[arm] = PcaPsf()
    pca_psf[arm].load(os.path.join(PSF_PATH.format(arm), 'pca.h5'))

    template_psf[arm] = PcaPsf.from_psf(gauss_psf[arm], grid.wave, size=s, truncate=5)
    # print(grid.wave.shape, 
    #     template_psf[arm].wave.shape, template_psf[arm].dwave.shape, template_psf[arm].pc.shape)

### Load sky, moon and conversion function

In [0]:
# Broad-band filter is common for all arms
filt_hsc_g = Filter()
filt_hsc_g.read(FILTER_PATH)

sky = {}
moon = {}

for arm in ARMS:
    detector_wave, _ = detector[arm].get_wave()
    detector_s = gauss_psf[arm].get_optimal_size(detector_wave)
    print(f'Optimal size of PSF kernel for arm {arm}', detector_s)
    detector[arm].psf = PcaPsf.from_psf(gauss_psf[arm], detector_wave, size=detector_s, truncate=5)

    sky[arm] = Sky()
    sky[arm].load(SKY_PATH.format(arm), format='h5')

    moon[arm] = Moon()
    moon[arm].load(MOON_PATH.format(arm), format='h5')

In [0]:
obs = {}

for arm in ARMS:
    obs[arm] = PfsObservation()
    obs[arm].detector = detector[arm]
    obs[arm].sky = sky[arm]
    obs[arm].moon = moon[arm]
    obs[arm].noise_model = NormalNoise()

### Create observation simulation pipeline

In [0]:
def create_pipeline(arm, grid, calib_bias=False):
    """
    Configure the observation simulation pipeline
    """

    pp = StellarModelPipeline()
    pp.model_res = grid.resolution or 150000
    pp.mag_filter = filt_hsc_g
    pp.observation = obs[arm]
    pp.snr = QuantileSnr(binning=1.0)
    pp.resampler = FluxConservingResampler()
    pp.noise_level = 1.0
    pp.noise_freeze = False
    if calib_bias:
        bias = FluxCalibrationBias(reuse_bias=False)
        bias.amplitude = 0.02
        pp.calibration = bias

    return pp

### Get high-resolution template

In [0]:
from pfs.ga.pfsspec.stellar.continuum.models import PiecewiseChebyshev
from pfs.ga.pfsspec.stellar.continuum.finders import SigmaClipping

In [0]:
def get_template(arm, convolve=True, continuum_normalize=False, wlim=None, **kwargs):
    """
    Generate a noiseless template spectrum with same line spread function as the
    observation but keep the original, high-resolution binning.
    """

    idx = grid.get_nearest_index(**kwargs)
    temp = grid.get_model_at(idx)
    temp.cont = None        # Make sure it's not passed around for better performance
    temp.mask = None

    if convolve:
        temp.convolve_psf(template_psf[arm])

    if wlim is not None:
        temp.trim_wave(wlim)

    
    cfit = PiecewiseChebyshev(deg=4, continuum_finder=SigmaClipping(max_iter=50, sigma=[1, 5]))
    cfit.init_wave(temp.wave)
    cparams = cfit.fit(temp)
    _, temp.cont = cfit.eval(cparams)

    if continuum_normalize:
        temp.normalize_by_continuum()

    return idx, temp

# Load X-Shooter data

In [0]:
from pfs.ga.pfsspec.survey.xsl import XslSpectrum, XslSurvey

In [0]:
xshooter = XslSurvey()
xshooter.load(os.path.join(XSHOOTER_PATH, 'xsl.dat'))

In [0]:
mask = (xshooter.params['Fe_H'] < -1.5) & (xshooter.params['log_g'] < 3.0) & (xshooter.params['T_eff'] > 4800)
print(mask.sum())
xshooter.params[mask]


In [0]:
XSLID = 79

xsl_spec = xshooter.spectra[XSLID]

print(xsl_spec.xsl_id)

f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

#mask = (790 <= xsl_spec.wave) & (xsl_spec.wave <= 796)
mask = (845 <= xsl_spec.wave) & (xsl_spec.wave <= 870)
#mask = np.full_like(spec.wave, True, dtype=np.bool)

ax.plot(xsl_spec.wave[mask], xsl_spec.flux[mask], '-k', lw=0.5)
ax.plot(xsl_spec.wave[mask], 10 * xsl_spec.flux_err[mask], '-r', lw=0.5)
if xsl_spec.flux_dered is not None:
    ax.plot(xsl_spec.wave[mask], xsl_spec.flux_dered[mask], '-b', lw=0.5)
if hasattr(xsl_spec, 'flux_sc') and xsl_spec.flux_sc is not None:
    ax.plot(xsl_spec.wave[mask], xsl_spec.flux_sc[mask], '-g', lw=0.5)

#ax.set_ylim(0, 1.5 * np.nanquantile(spec.flux[mask], 0.99))
ax.set_ylim(0.0e-12, 1.35e-12)

ax.set_xlabel('wavelength [nm]')
ax.set_ylabel(r'$F_\lambda$ [erg/s/cm2/A]')

ax.set_title('{} \n[Fe/H]={}, T_eff={}, log_g={}'.format(xshooter.params.iloc[XSLID]['obj_id'], xsl_spec.Fe_H, xsl_spec.T_eff, xsl_spec.log_g))

In [0]:
xshooter.params.iloc[XSLID]

## Degrade resolution

In [0]:
# X-Shooter sigma

def xsl_sigma(wave):
    fwhm = np.full_like(wave, np.nan)

    masks = [
        (3200 <= wave) & (wave < 5500),
        (5500 <= wave) & (wave < 10000),
        (10000 <= wave) & (wave < 25000)
    ]
    res = [ 9793, 11573, 7956 ]

    for mask, r in zip(masks, res):
        fwhm[mask] = wave[mask] / r

    return fwhm / (2 * np.sqrt(2 * np.log(2)))

In [0]:
deg_psf = {}
for arm in ARMS:
    psf = GaussPsf(orig=gauss_psf[arm])
    psf.sigma = np.sqrt(gauss_psf[arm].sigma**2 - xsl_sigma(gauss_psf[arm].wave)**2)
    deg_psf[arm] = psf

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

for arm in ARMS:
    ax.plot(gauss_psf[arm].wave, gauss_psf[arm].sigma, lw=0.5)
    ax.plot(gauss_psf[arm].wave, deg_psf[arm].sigma, '--k', lw=0.5)

ax.grid()

ax.set_xlabel('wavelength [A]')
ax.set_ylabel('LSF sigma')

In [0]:
deg_psf

In [0]:
# Reconfigure obs simulation to use resolution degrading kernel

for arm in ARMS:
    detector_wave, _ = detector[arm].get_wave()
    detector_s = deg_psf[arm].get_optimal_size(detector_wave)
    print(f'Optimal size of PSF kernel for arm {arm}', detector_s)
    detector[arm].psf = PcaPsf.from_psf(deg_psf[arm], detector_wave, size=detector_s, truncate=5)

In [0]:
from pfs.ga.pfsspec.core.util.copy import safe_deep_copy

xsl_spec = xshooter.spectra[XSLID]

xsl_degraded = {}
for arm in ARMS:
    xsl_nspec = type(xsl_spec)()
    xsl_nspec.wave = safe_deep_copy(xsl_spec.wave) * 10
    #nspec.wave_edges = safe_deep_copy(spec.wave_edges)
    xsl_nspec.flux = safe_deep_copy(xsl_spec.flux)
    xsl_nspec.flux_err = safe_deep_copy(xsl_spec.flux_err)
    xsl_nspec.flux_dered = safe_deep_copy(xsl_spec.flux_dered)

    xsl_nspec.normalize_to_mag(filt_hsc_g, 22)

    res = FluxConservingResampler()
    wave, wave_edges = detector[arm].get_wave()

    xsl_nspec.convolve_psf(deg_psf[arm])
    xsl_nspec.apply_resampler(res, wave, wave_edges)

    xsl_degraded[arm] = xsl_nspec

### Get matching template

In [0]:
xsl_spec.Fe_H, xsl_spec.T_eff, xsl_spec.log_g, 

In [0]:
grid.array_grid.axes['a_M'].values

In [0]:
star_HD9051 = params = {
    'M_H': -2.0, #xsl_spec.Fe_H,
    'T_eff': xsl_spec.T_eff,
    'log_g': xsl_spec.log_g,
    'a_M': 1.0
}

print(params)

temp_degraded = {}
for arm in ARMS:
    _, temp_degraded[arm] = get_template(arm, convolve=True, continuum_normalize=False, wlim=None, **params)

In [0]:
temp_degraded['mr'].M_H, temp_degraded['mr'].T_eff, temp_degraded['mr'].log_g, temp_degraded['mr'].a_M

In [0]:
# xsl_nspec = type(xsl_spec)()
# xsl_nspec.wave = safe_deep_copy(xsl_spec.wave) * 10
# #nspec.wave_edges = safe_deep_copy(spec.wave_edges)
# xsl_nspec.flux = safe_deep_copy(xsl_spec.flux)
# xsl_nspec.flux_err = safe_deep_copy(xsl_spec.flux_err)
# xsl_nspec.flux_dered = safe_deep_copy(xsl_spec.flux_dered)

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

ax.plot(xsl_spec.wave * 10, xsl_spec.flux * 0.8e-5, '-k', lw=0.5, label='XSL')
#ax.plot(xsl_degraded['mr'].wave, xsl_degraded['mr'].flux * 0.5e-5, '-r', lw=0.5, label='XSL degraded')
ax.plot(xsl_degraded['mr'].wave, xsl_degraded['mr'].flux, '-r', lw=0.5, label='XSL degraded')
ax.plot(temp_degraded['mr'].wave, temp_degraded['mr'].flux * 0.3e-31, '-b', lw=0.5, label='model at PSF resolution')

ax.set_xlim(8490, 8555)
ax.set_ylim(0, 1.2e-17)
ax.set_title('XSL spectrum {} - mr'.format(xshooter.params.iloc[XSLID]['obj_id']))
ax.legend()

In [0]:
for arm in ARMS:
    f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

    ax.plot(xsl_spec.wave * 10, xsl_spec.flux * 0.8e-5, '-k', lw=0.5)
    #ax.plot(xsl_spec.wave * 10, xsl_spec.flux_err, '-k', lw=0.5, label='original XSL')

    ax.plot(xsl_degraded[arm].wave, xsl_degraded[arm].flux, '-r', lw=0.5)
    #ax.plot(xsl_degraded[arm].wave, xsl_degraded[arm].flux_err, '-r', lw=0.5, label='degraded to PFS')

    ax.plot(temp_degraded[arm].wave, temp_degraded[arm].flux * 0.3e-31, '-b', lw=0.5, label='model at PSF resolution')

    ax.set_xlim(xsl_degraded[arm].wave.min(), xsl_degraded[arm].wave.max())
    ax.set_ylim(0, 1.2e-17)
    ax.set_title(f'{xshooter.params.iloc[XSLID]["obj_id"]} XSL spectrum - {arm}')
    ax.legend()

# Fit RV

### Stellar parameters

In [0]:
for i, k, ax in grid.enumerate_axes():
    print(k, ax.values)

In [0]:
# dSph RGB star
star_dSph_RGB = {
    'M_H': -1.5,
    'T_eff': 5000,
    'log_g': 3.0,
    'a_M': 0.0
}

star_HD2796 = {
    'M_H': -2.5,
    'T_eff': 4700,
    'log_g': 1.5,
    'a_M': 0.0
}

star_dSph_RGB_off = [star_dSph_RGB,]
for M_H in [-2, -1]:
    for T_eff in [4800, 5200]:
        for log_g in [2.5, 3.5]:
            star_dSph_RGB_off.append({
                'M_H': M_H,
                'T_eff': T_eff,
                'log_g': log_g,
                'a_M': 0.0
            })

print(len(star_dSph_RGB_off))

### Observation parameters

In [0]:
obs_dark_optimal = {
    'seeing': 0.5,
    'exp_time': 15 * 60,
    'exp_count': 4 * 3,
    'target_zenith_angle': 0,
    'target_field_angle': 0.0,
    'moon_zenith_angle': 45,
    'moon_target_angle': 60,
    'moon_phase': 0.,
    'sky_residual': 0.00,
}

### Configure RV fit simulation

In [0]:
def get_rvfitsim(model_params, temp_params, model_spectrum=None, normalize=False):
    # When obs is None, sample from a model grid
    # Else, sample the noise based on the ETC model but use obs

    rvfitsim = ModelGridRVFitSim(random_state=np.random.RandomState(None))
    rvfitsim.arms = [ 'b', 'mr', 'n' ]

    rvfitsim.threads = 12
    rvfitsim.parallel = True

    rvfitsim.rvfit = RVFit()
    # rvfitsim.rv_bounds = (100, 300)
    rvfitsim.rv_bounds = (-200, 100)

    # Override grid sampling with a pre-defined model spectrum
    rvfitsim.model_spectrum = model_spectrum        # Original resolution observation from X-Shooter
    rvfitsim.model_params = model_params

    rvfitsim.template_grid = grid
    rvfitsim.template_params = temp_params
    rvfitsim.template_psf = {}                              # No pre-fit resolution degrading

    if normalize:
            rvfitsim.continuum_model = PiecewiseChebyshev(deg=4, continuum_finder=SigmaClipping(max_iter=50, sigma=[1, 5]))
            rvfitsim.continuum_normalize = True

    rvfitsim.observation_params = obs_dark_optimal

    sampler = ModelGridSampler()
    sampler.grid = grid
    # Prevent additional shifting
    sampler.add_parameter(Parameter('rv', value=0, dist='const'))
    
    rvfitsim.set_sampler(sampler)

    rvfitsim.observation_snr = QuantileSnr(binning=1.0)
    rvfitsim.observation_pipeline = {}

    for arm in rvfitsim.arms:
        rvfitsim.template_psf[arm] = template_psf[arm]      # PSF sampled at model resolution
        rvfitsim.observation_pipeline[arm] = create_pipeline(arm, grid, calib_bias=False)

    return rvfitsim


In [0]:
from pfs.ga.pfsspec.stellar.rvfit import RVFitTrace

In [0]:
from pfs.ga.pfsspec.core import physics
from pfs.ga.pfsspec.sim.rvfit import RVFitSimTrace
from pfs.ga.pfsspec.sim.rvfit import ModelGridRVFitSimTrace

class Trace(ModelGridRVFitSimTrace, RVFitTrace):
    def on_get_template(self, arm, temp):
        f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)
        ax.plot(temp.wave, temp.flux, lw=0.5)
        if temp.cont is not None:
            ax.plot(temp.wave, temp.cont, lw=0.5)

        ax.set_ylim(0, None)

        ax.set_title('Template')

    def on_get_model(self, arm, spec):
        pass

    def on_generate_params(self, params):
        for k, v in params.items():
            print(k, v)

    def on_generate_observation(self, arm, spec, params):
        pass

    def on_generate_noise(self, arm, spec):
        f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)
        # ax.plot(spec.wave, spec.flux / spec.flux.max(), lw=0.5, label='noisy flux')
        # ax.plot(spec.wave, spec.flux_model / spec.flux_model.max(), lw=0.5, label='model flux')
        # ax.plot(spec.wave, spec.cont / spec.cont.max(), lw=0.5, label='continuum')
        # ax.plot(spec.wave, spec.cont_model / spec.cont_model.max(), lw=0.5, label='model cont')

        ax.plot(spec.wave, spec.flux, lw=0.5, label='flux')
        
        if spec.mask is not None:
            ax.plot(spec.wave[spec.mask], np.full_like(spec.wave[spec.mask], spec.flux.mean()), '.', ms=1)

        ax.set_ylim(0, None)

        ax.grid()
        ax.legend()
        ax.set_title(f'Noisy model in arm {arm}')

    def on_normalize_continuum(self, arm, spec):
        f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

        ax.plot(spec.wave, spec.flux, lw=0.5, label='normalized flux')

        ax.set_ylim(0, None)

        ax.grid()
        ax.legend()
        ax.set_title(f'Normalized model in arm {arm}')

    def on_guess_rv(self, rv, log_L, fit, function, pp, pcov):
        f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

        ax.plot(rv, log_L)
        ax.plot(rv, fit)

        ax.set_title('Guess RV')

    def on_fit_rv(self, rv, spec, temp):
        for s, t in zip(spec, temp):
            f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

            ax.plot(s.wave, s.flux / np.mean(s.flux), lw=0.3)
            ax.plot(t.wave, t.flux / np.mean(t.flux), lw=0.1)

            ax.set_ylim(0, None)

            ax.set_title(f'Best fit: {rv:.2f} km/s')

            # CaII triplet
            if (s.wave[0] < 8450) and (8700 < s.wave[-1]):
                for lw in [8498, 8542, 8662]:
                    lw = Physics.air_to_vac(lw)

                    mask = (lw * (1 + s.redshift) - 15 <= s.wave) & (s.wave <= lw * (1 + s.redshift) + 15)

                    f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

                    ax.plot(s.wave[mask], s.flux[mask] / np.mean(s.flux[mask]), lw=0.5)
                    ax.plot(t.wave[mask], t.flux[mask] / np.mean(t.flux[mask]), lw=0.3)

                    ax.axvline(lw * (1 + s.redshift), c='k', lw=0.5)
                    ax.axvline(lw, c='k', ls='--', lw=0.5)
                    ax.axvline(lw * (1 + Physics.vel_to_z(rv)), c='r', lw=0.5)

                    ax.set_title(f'Ground truth: {Physics.z_to_vel(s.redshift):.2f} km/2, best fit: {rv:.2f} km/s')

In [0]:
# Set logging level to error to absorb warnings
from pfs.ga.pfsspec.core.util.dict import safe_deep_compare

xsl_nspec = type(xsl_spec)()
xsl_nspec.wave = safe_deep_copy(xsl_spec.wave) * 10
#nspec.wave_edges = safe_deep_copy(spec.wave_edges)
xsl_nspec.flux = safe_deep_copy(xsl_spec.flux)
xsl_nspec.flux_err = safe_deep_copy(xsl_spec.flux_err)
xsl_nspec.flux_dered = safe_deep_copy(xsl_spec.flux_dered)


l = logging.getLogger()
l.setLevel(logging.ERROR)

rv_gt = {}
rv_fit = {}
rv_err = {}
rv_snr = {}

mc_count = 10
repeat = 20

mags = [19, 20, 21, 22, 23]
calib_bias = [False]
normalize = False

n = mc_count * len(mags) #* len(star_dSph_RGB_off)
t = tqdm(total=n)

def do():
    for m in mags:
        rv_gt[m] = {}
        rv_fit[m] = {}
        rv_err[m] = {}
        rv_snr[m] = {}

        for cb in calib_bias:
            rv_gt[m][cb] = []
            rv_fit[m][cb] = []
            rv_err[m][cb] = []
            rv_snr[m][cb] = []

            #for temp_args in star_dSph_RGB_off:
            #for temp_args in [star_dSph_RGB]:
            for temp_args in [star_HD9051]:
                params = {
                    'mag': m,
                    #**star_dSph_RGB
                    **star_HD9051
                }
                #rvfitsim = get_rvfitsim(params, temp_args, normalize=normalize)
                rvfitsim = get_rvfitsim(params, temp_args, model_spectrum=xsl_nspec, normalize=normalize)
                
                # rvfitsim.parallel = False
                # rvfitsim.trace = Trace()
                # rvfitsim.rvfit.trace = Trace()
                
                res = rvfitsim.rvfit_mc(mc_count=mc_count, repeat=repeat, tqdm=t)
                for c, r in zip((rv_gt[m][cb], rv_fit[m][cb], rv_err[m][cb], rv_snr[m][cb]), res):
                    c.append(r)

                #return

do()

In [0]:
# from collections.abc import Iterable
# import pickle

# fn = 'rvfit'
# fn += '_' + '_'.join(a for a in rvfitsim.arms)
# fn += '_m' + '_'.join(str(m) for m in rv_fit.keys())
# if isinstance(rvfitsim.observation_rv, Iterable):
#     fn += '_rv' + '_'.join(str(rv) for rv in rvfitsim.observation_rv)
# else:
#     fn += '_rv' + str(rvfitsim.observation_rv)
# fn += '.dat'

# print(fn)

# with open(fn, 'wb') as f:
#     pickle.dump([rv_gt, rv_fit, rv_err, rv_snr], f)

In [0]:
#bins = np.linspace(-2, 2, 20)

#bins = np.linspace(-12, 12, 20)
bins = np.linspace(-95, -65, 20)

f, axs = plt.subplots(1, len(rv_fit), figsize=(7, 2), dpi=240, squeeze=False)

for i, m in enumerate(rv_fit):
    ax = axs[0, i]
    for cb in rv_fit[m]:
        for j in reversed(range(len(rv_fit[m][cb]))):
            hist, _ = np.histogram(rv_fit[m][cb][j].flatten() - rv_gt[m][cb][j].flatten(),
                                   bins=bins, density=True)
            ax.step(0.5 * (bins[1:] + bins[:-1]), hist, where='mid',
                ls='-' if j == 0 else '--', lw=0.5,
                c='r' if j == 0 else 'k',
                label="{} flux bias".format('with' if cb else 'no'))

    # ax.axvline(rv, c='r', label="ground truth")

    ax.set_xlabel(r'$\Delta\,$RV [km s$^{-1}]$')
    ax.set_ylabel('')

    snr = [np.nanmean(rv_snr[m][False][0][k]) for k in rv_snr[m][False][0]]
    snr = ','.join([f'{s:.0f}' for s in snr])
    ax.set_title(f"mag = {m:.2f}\nsnr = {snr}")

    ax.grid()

for ax in axs.flatten():
    ax.set_ylim(0, 0.5)

for ax in axs[1:].flatten():
    ax.yaxis.set_ticklabels([])

f.suptitle(f"arms: {', '.join(FIT_ARMS)}")

f.tight_layout()

In [0]:
#rv_gt
#rv_fit

rv_bias = {}
rv_std = {}
for m in rv_fit:
    rv_bias[m] = {}
    rv_std[m] = {}
    for cb in rv_fit[m]:
        rv_bias[m][cb] = {}
        rv_std[m][cb] = {}
        for k in range(len(rv_fit[m][cb])):
            d = rv_fit[m][cb][k] - (-78.1) #rv_gt[m][cb][k]
            rv_bias[m][cb][k] = np.nanmean(d)
            rv_std[m][cb][k] = np.nanstd(d)


In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

m0 = list(rv_std.keys())[0]
cb = False

for k in reversed(range(len(rv_std[m][cb]))):
    ax.plot([m for m in rv_std], [rv_std[m][False][k] for m in rv_std], 
        ls='-' if k == 0 else '--', 
        lw=1 if k == 0 else 0.3, 
        c='red',
        label='uncertainty from simulations' if k == 0 else None)
    ax.plot([m for m in rv_bias], [rv_bias[m][False][k] for m in rv_bias], 
        ls='-' if k == 0 else '--', 
        lw=1 if k == 0 else 0.3,
        c='green',
        label='bias from simulations' if k == 0 else None)
    ax.plot([m for m in rv_err], [rv_err[m][False][k].mean() for m in rv_err],
        ls='-' if k == 0 else '--', 
        lw=1 if k == 0 else 0.3,
        c='blue',
        label='error from Fisher matrix' if k == 0 else None)

ax.set_xlabel('mag g')
ax.set_ylabel(r'$\Delta RV$')
ax.set_title(f"arms: {', '.join(FIT_ARMS)}")
ax.legend()
ax.grid()